In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [3]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [4]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [5]:
for d in datasets:
    print(d.name)



df_enhanced.csv
df_enhanced.csv_scaled.csv
df_enhanced.csv_selected_features.csv
df_enhanced.csv_selected_features.csv_scaled.csv
df_enhanced_one-hot.csv
df_enhanced_one-hot.csv_scaled.csv
df_enhanced_one-hot.csv_selected_features.csv
df_enhanced_one-hot.csv_selected_features.csv_scaled.csv
V4_classification_lr3.csv
V4_classification_lr3.csv_scaled.csv


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Input, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import xgboost as xgb    

def create_mlp_model(input_shape, num_classes, optimizer='adam', learning_rate=0.001, dense_units=[128, 64]):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_1d_cnn_model(input_shape, num_classes, filters=[128, 256],
                         kernel_size=3, pool_size=2, use_batch_norm=True, dense_units=[128, 64],
                         optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(input_shape, 1))) # input_shape уже должен быть (num_features, 1)
    i=0
    for f in filters:
        model.add(Conv1D(filters=f, kernel_size=kernel_size, activation='relu', padding='same'))
        if use_batch_norm:
            model.add(BatchNormalization())
        # Добавляем MaxPooling1D, если есть смысл и достаточное количество признаков
        if input_shape > pool_size * (i + 1) : # Убедимся, что размерность не станет слишком маленькой
            model.add(MaxPooling1D(pool_size=pool_size))
        else:
            # Если MaxPooling1D может сделать размерность 0, пропускаем его
            # или можно настроить логику по-другому, например, убрать padding='same'
            # и добавить его только на первом слое
            pass # Пропускаем MaxPooling, если он приведет к 0 размерности
        i+=1
    model.add(Flatten())
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
        if use_batch_norm: # Можно добавить BatchNormalization и здесь, но это опционально
            model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))

    # Выбор оптимизатора
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate) # Default

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_gr_b(n_estimators, learning_rate, max_depth, random_state, dtrain, dtest):
    params = {
        'objective': 'multi:softmax',  # Бинарная классификация

    'num_class': 7,
    'eval_metric': 'mlogloss',
        
        'learning_rate': learning_rate,           # Скорость обучения
        'max_depth': max_depth,                  # Глубина деревьев
        'subsample': 0.9,                
        'colsample_bytree': 0.8,         
        'alpha': 0.1,                    # L1 регуляризация
        'lambda': 1,                     # L2 регуляризация
        'seed': random_state,                      # Для воспроизводимости
        'early_stopping_rounds': 10      # Ранняя остановка
    }

    # Обучение модели
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=n_estimators,             # Максимальное число деревьев
        evals=[(dtest, 'eval')],          # Данные для валидации
        verbose_eval=10                  # Вывод каждые 10 итераций
    )
    return model, params
    

In [7]:
from itertools import product

def all_combinations(args_dict):
    # Получаем имена аргументов и их возможные значения
    arg_names = args_dict.keys()
    values = args_dict.values()
    res = []
    # Перебираем все комбинации значений
    for combination in product(*values):
        # Создаем словарь с аргументами
        kwargs = dict(zip(arg_names, combination))
        # Вызываем функцию с текущей комбинацией
        res.append(kwargs)
    return res



In [8]:
import matplotlib.pyplot as plt
from xgboost import plot_importance
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

for d in datasets:

    
    num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов
    assert num_classes == 7
    #temp = []
    #for i in range(num_classes):
    #    temp.append(d.target_features[0] + f"_{i}")
    #d.target_features = 
    #d[temp] = to_categorical(d[d.target_features].values - 1, num_classes=7) # Эта строка остается
    #d = d.drop(d.target_features, axis=1)
    #d = dp.Dataset(d)
    #d.target_features = temp
    #d.name = temp_name
    d[d.target_features] -= 1
    d.split_data(random_state=1337)
    dtrain = xgb.DMatrix(d.X_train.values, label=d.Y_train.values)
    dtest = xgb.DMatrix(d.X_test.values, label=d.Y_test.values)


   
    params_tree = {"n_estimators" : [100, 50, 20, 200], "learning_rate": [0.1, 0.2, 0.05], "max_depth": [3, 4, 5], "random_state": [1337]}
    default_params = {"dtrain" : [dtrain] , "dtest": [dtest]}
    tree_comb = all_combinations({**params_tree, **default_params })

    #print(len(mlp_comb))
    print(len(tree_comb))

    iii = 0
    #for el in mlp_comb:
    for el in tree_comb:

        print(iii, el)
        iii+=1
        #model = create_mlp_model(**el)
        #model = create_1d_cnn_model(**el)

        model, params = create_gr_b(**el)
        y_pred_prob = model.predict(dtest)    # Вероятности класса
        y_pred = y_pred_prob  # Преобразование в метки
        #print(np.sum(y_pred_prob))
        #print(np.sum(d.Y_test.values))
        #print((d.Y_test.to_numpy()))
        #y_pred = y_pred_prob
        y_real = d.Y_test.values  # Преобразование в метки 
        with open('results_tree.txt', 'a', encoding='utf-8') as f:
            f.write(f"{d.name} {el}\n")

            # Оценка модели
            f.write("\nТочность: {:.4f}".format(accuracy_score(y_real, y_pred)))
            f.write("\nОтчет классификации:")
            f.write(f"{classification_report(y_real, y_pred)}")
            
            # Матрица ошибок
            f.write("\nМатрица ошибок:")
            f.write(f"{confusion_matrix(y_real, y_pred)}")
        




C:\Users\nikol\AppData\Local\Temp\ipykernel_14832\3180493565.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов


36
0 {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3, 'random_state': 1337, 'dtrain': <xgboost.core.DMatrix object at 0x0000000024576750>, 'dtest': <xgboost.core.DMatrix object at 0x00000000238BE870>}
[0]	eval-mlogloss:1.77507


S:\anaconda\envs\study\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:02] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "early_stopping_rounds" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[10]	eval-mlogloss:1.05767
[20]	eval-mlogloss:0.82547
[30]	eval-mlogloss:0.71985
[40]	eval-mlogloss:0.66459
[50]	eval-mlogloss:0.63000
[60]	eval-mlogloss:0.60714
[70]	eval-mlogloss:0.59080
[80]	eval-mlogloss:0.57846
[90]	eval-mlogloss:0.56933
[99]	eval-mlogloss:0.56158
1 {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 4, 'random_state': 1337, 'dtrain': <xgboost.core.DMatrix object at 0x0000000024576750>, 'dtest': <xgboost.core.DMatrix object at 0x00000000238BE870>}
[0]	eval-mlogloss:1.76742


S:\anaconda\envs\study\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:07] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "early_stopping_rounds" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

[10]	eval-mlogloss:1.34467
[20]	eval-mlogloss:1.09012
[30]	eval-mlogloss:0.93819
[40]	eval-mlogloss:0.84047
[50]	eval-mlogloss:0.77641
[60]	eval-mlogloss:0.73085
[70]	eval-mlogloss:0.69797
[80]	eval-mlogloss:0.67310
[90]	eval-mlogloss:0.65344
[100]	eval-mlogloss:0.63758
[110]	eval-mlogloss:0.62469
[120]	eval-mlogloss:0.61388
[130]	eval-mlogloss:0.60519
[140]	eval-mlogloss:0.59766
[150]	eval-mlogloss:0.59165
[160]	eval-mlogloss:0.58650
[170]	eval-mlogloss:0.58169
[180]	eval-mlogloss:0.57720
[190]	eval-mlogloss:0.57349
[199]	eval-mlogloss:0.57001
34 {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 4, 'random_state': 1337, 'dtrain': <xgboost.core.DMatrix object at 0x000000002461E210>, 'dtest': <xgboost.core.DMatrix object at 0x000000002461CF50>}
[0]	eval-mlogloss:1.85515


S:\anaconda\envs\study\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:43:45] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "early_stopping_rounds" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[10]	eval-mlogloss:1.32180
[20]	eval-mlogloss:1.05835
[30]	eval-mlogloss:0.90137
[40]	eval-mlogloss:0.80076
[50]	eval-mlogloss:0.73467
[60]	eval-mlogloss:0.68771
[70]	eval-mlogloss:0.65336
[80]	eval-mlogloss:0.62746
[90]	eval-mlogloss:0.60669
[100]	eval-mlogloss:0.59025
[110]	eval-mlogloss:0.57732
[120]	eval-mlogloss:0.56698
[130]	eval-mlogloss:0.55808
[140]	eval-mlogloss:0.55056
[150]	eval-mlogloss:0.54375
[160]	eval-mlogloss:0.53821
[170]	eval-mlogloss:0.53314
[180]	eval-mlogloss:0.52837
[190]	eval-mlogloss:0.52478
[199]	eval-mlogloss:0.52108
35 {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 5, 'random_state': 1337, 'dtrain': <xgboost.core.DMatrix object at 0x000000002461E210>, 'dtest': <xgboost.core.DMatrix object at 0x000000002461CF50>}
[0]	eval-mlogloss:1.85131


S:\anaconda\envs\study\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:44:00] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "early_stopping_rounds" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[10]	eval-mlogloss:1.30052
[20]	eval-mlogloss:1.02803
[30]	eval-mlogloss:0.86564
[40]	eval-mlogloss:0.76149
[50]	eval-mlogloss:0.69318
[60]	eval-mlogloss:0.64566
[70]	eval-mlogloss:0.61044
[80]	eval-mlogloss:0.58346
[90]	eval-mlogloss:0.56206
[100]	eval-mlogloss:0.54523
[110]	eval-mlogloss:0.53200
[120]	eval-mlogloss:0.52096
[130]	eval-mlogloss:0.51169
[140]	eval-mlogloss:0.50443
[150]	eval-mlogloss:0.49747
[160]	eval-mlogloss:0.49155
[170]	eval-mlogloss:0.48609
[180]	eval-mlogloss:0.48079
[190]	eval-mlogloss:0.47713
[199]	eval-mlogloss:0.47352


In [14]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
